In [29]:
import json
import numpy as np
import re
from docx import Document 
import docx
import os

Třída pro parsování Markdown položek, pro shlukování textu pro lepší překlady

In [30]:
class markdown_text:
    def __init__(self, lines_orig):
        self.lines_orig = lines_orig
    
    def parse_orig(self):
        self.lines_CZ = list()
        first_line = True
        tmp_line = ''
        for line in self.lines_orig:
            if line[0] == '#' or line[0:3] == ' - ' or line[0:2] == ' *' or line[0:1] == '*' or line[0:2] == '- ' or line[0:6] == '    - ':
                if tmp_line != '':
                    self.lines_CZ.append(tmp_line)
                self.lines_CZ.append(line.replace('\n',''))
                first_line = True
                tmp_line = ''
            else:
                if first_line:
                    first_line = False
                    tmp_line = line.replace('\n','')
                else:
                    tmp_line = tmp_line + ' ' + line.replace('\n','')
                    
        if tmp_line != '':
            self.lines_CZ.append(tmp_line)
            
        self.n_lines = len(self.lines_CZ)
    
    def set_trans(self, lines_translated):
        for i in range(self.n_lines):
            line_CZ = self.lines_CZ[i]
            line_EN = lines_translated[i]
            formulas_CZ = re.findall("\$(.*?)\$",line_CZ)
            formulas_EN = re.findall("\$(.*?)\$",line_EN)
            for j in range(len(formulas_EN)):
                line_EN = line_EN.replace('$'+formulas_EN[j]+'$','$'+formulas_CZ[j]+'$')
            lines_translated[i] = line_EN + ' \n'
        self.lines_EN = lines_translated
        return self.lines_EN
            

Projdu všechny soubory v složce cz

In [31]:
all_files = os.listdir(path='cz/')
all_files_n = list()
for line in all_files:
    all_files_n.append(line[:-6])


Zpracujeme soubory

In [32]:
document = Document()

In [33]:
for filename in all_files_n:
    document.add_paragraph('%%%%%%%%%% ' + filename + ' \n')
    json_file = open('cz/'+filename + '.ipynb')
    data = json.load(json_file)
    all_cells = data['cells']
    mark_classes = list()
    out_list = list()
    n_lines = 0

    for one_cell in all_cells:
        if one_cell['cell_type'] == 'markdown':
            mt= markdown_text(one_cell['source'].copy())
            mt.parse_orig()
            out_lines = mt.lines_CZ
            out_list = out_list + out_lines
            n_lines += mt.n_lines

        if one_cell['cell_type'] == 'code':
            for line in one_cell['source']:
                idx = line.find('#')
                if idx > -1:
                    out_list.append(line[(idx+1):].replace('\n',''))
                    n_lines += 1

    for i in range(len(out_list)):
        out_list[i] = 'L' + str(i) + ': ' + out_list[i] + '\n'

    for line in out_list:
        document.add_paragraph(line)

In [34]:
document.save('to_translate.docx')

In [35]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return fullText

V tuto chvíli je třeba vyrobený soubor, přeložit pomocí Google translate!!!!

In [36]:
translated_lines = getText('translated.docx')

In [37]:
all_files = list()
for line in translated_lines:
    if line[:11] == '%%%%%%%%%% ':
        this_file = list()
        all_files.append(this_file)
    else:
        bold_txt = re.findall("\*\*(.*?)\*\*",line)
        tmp = line
        for bold_piece in bold_txt:
            tmp = tmp.replace(bold_piece,bold_piece[1:-1])
        tmp = tmp.replace(' =','=').replace('= ','=').replace(' (','(').replace(' |','|').replace('| ','|').replace(' >','>').replace('> ','>').replace(' <','<').replace('< ','<').replace(' ~','~').replace('~ ','~')
        tmp = tmp.replace('! [',' </br> \n ![').replace('attachment: ','attachment:').replace(' *(','*(').replace(' /','/').replace('/ ','/').replace(' .','.')
        idx = tmp.find(':')
        tmp = tmp[(idx+2):]
        this_file.append(tmp)


Projde řádky a vyrobí nové ipynb

In [38]:
idx_file = 0
for filename in all_files_n:
    translated_single_file = all_files[idx_file]
    idx_file += 1
    json_file = open('cz/'+filename + '.ipynb')
    data = json.load(json_file)
    all_cells = data['cells']
    mark_classes = list()
    out_list = list()
    n_lines = 0

    for one_cell in all_cells:
        if one_cell['cell_type'] == 'markdown':
            mt= markdown_text(one_cell['source'].copy())
            mt.parse_orig()
            out_lines = mt.set_trans(translated_single_file[n_lines:(n_lines+mt.n_lines)])
            one_cell['source'] = out_lines
            n_lines += mt.n_lines

        if one_cell['cell_type'] == 'code':
            i = 0
            for line in one_cell['source']:
                idx = line.find('#')
                if idx > -1:
                    out_list.append(line[(idx+1):].replace('\n',''))
                    one_cell['source'][i] = line[0:(idx+1)] + ' ' + translated_single_file[n_lines]
                    n_lines += 1
                i += 1

                
    for one_cell in all_cells:
        if one_cell['cell_type'] == 'markdown' or one_cell['cell_type'] == 'code':
            i = 0
            for line in one_cell['source']:
                tmp = line.replace('variace_opak','r_permutation_repetition').replace('variace','r_permutation')
                tmp = tmp.replace('citatel','numerator').replace('jmenovatel','denominator')
                tmp = tmp.replace('permutace_opak','permutation_repetition').replace('permutace','permutation')
                tmp = tmp.replace('kombinace_opak','combinations_repetition').replace('kombinace','combinations').replace('pocet','count')               
                one_cell['source'][i] = tmp
                i += 1
    
    
    outfile = open('en/'+filename + '.ipynb', 'w')
    json.dump(data, outfile)
    outfile.close()
